<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=588d08a03857cbc1a6722ed1de2d1818ae098c93425d372bb13ad1e3742bbca8
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-16 09:43:08
-------------------
qualified stocks: 86
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  1.09 L
Current:  1.40 C
-------------------
Today PnL: 72.45 K (0.52%)
Current PnL: -21.28 L (-14.18%)
CY Booked + Current PnL: -9.38 L (-6.25%)
-------------------
Total profit:  1.73 L
Total loss:  -23.01 L
-------------------
Total Booked + Current PnL: 17.60 L (14.29%)
Total Booked PnL: 38.88 L (31.58%)
Curr Year Booked PnL: 11.90 L (8.49%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 88.47 L (63.1%)
Deployed:  1.23 C
Current:  1.40 C
CAGR/XIRR %: 7.73%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CIPLA,1662.74,-19.78,58.0,X-LC,3.78,214405.0,9905.0,13400.0,0.40,4.84,6.25,11.39,10.0,0.74,1.54,14.23,X40N,ATH,PHARMA
76,TTKPRESTIG,770.00,96.80,50.0,M-SC,1.65,84744.0,-16033.0,16127.0,0.54,-15.91,19.03,0.09,245.0,-0.99,0.61,11.33,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.88,57.0,X-MC,1.25,211695.0,19953.0,17804.0,-0.06,10.41,8.41,19.69,99.0,1.12,1.52,17.91,XY25,NTT,AC
33,ICICIGI,2260.25,-13.38,67.0,X-MC,2.08,149110.0,12977.0,18147.0,-0.26,9.53,12.17,22.86,91.0,0.72,1.07,23.35,X40,ATH,INSURANCE
56,RELIANCE,1533.00,-14.04,49.0,X-LC,2.85,215218.0,4852.0,23932.0,0.39,2.31,11.12,13.68,37.0,0.20,1.55,19.21,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.40,-18.06,43.0,M-SC,25.55,78942.0,-97873.0,178669.0,3.06,-55.35,226.33,45.70,197.0,-0.55,0.57,3.06,XY24,NTT,TRAVEL
66,SONACOMS,1006.00,-34.58,68.0,M-SC,12.32,81506.0,-19653.0,94547.0,2.98,-19.43,116.00,74.03,202.0,-0.21,0.59,14.93,AR,BTT,AUTO
84,WHIRLPOOL,2270.00,-46.09,47.0,M-SC,4.29,91200.0,-298.0,79052.0,2.79,-0.33,86.68,86.07,223.0,-0.00,0.66,32.44,XR,NTT,DURABLES
63,SHALBY,327.00,1179.11,55.0,M-SC,22.97,173874.0,-7269.0,52736.0,2.47,-4.01,30.33,25.10,235.0,-0.14,1.25,39.34,XY24,NTT,HEALTHCARE
26,HAPPSTMNDS,1488.71,-25.15,34.0,H-SC,15.14,80704.0,-47461.0,157486.0,2.19,-37.03,195.14,85.85,132.0,-0.30,0.58,3.02,AR,ATH,IT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
27,HATHWAY,31.4,863.77,39.0,H-SC,5.16,82980.0,-25740.0,105418.0,-0.93,-23.68,127.04,73.29,161.0,-0.24,0.60,11.00,XR,NTT,ENTERTAINMENT
81,VALIANTORG,838.0,-282.78,31.0,H-SC,4.17,101660.0,-61945.0,183262.0,-0.78,-37.86,180.27,74.15,139.0,-0.34,0.73,30.31,XR,NTT,CHEMICALS
42,JCHAC,2282.0,19704.35,46.0,M-SC,1.67,96762.0,-31043.0,31032.0,-0.64,-24.29,32.07,-0.01,232.0,-1.00,0.70,9.57,OX40N,NTT,AC
39,INFY,2275.0,-19.96,45.0,X-LC,7.55,312045.0,-843.0,172530.0,-0.64,-0.27,55.29,54.87,3.0,-0.00,2.24,6.36,X40,BTT,IT
15,CAMPUS,393.0,-22.64,60.0,M-SC,2.81,161027.0,-6748.0,62591.0,-0.61,-4.02,38.87,33.28,210.0,-0.11,1.16,24.81,XY24,NTT,FOOTWEAR


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-46.09,47.0,M-SC,4.29,91200.0,-298.0,79052.0,2.79,-0.33,86.68,86.07,223.0,-0.0,0.66,32.44,XR,NTT,DURABLES
60,SAMMAANCAP,326.0,-186.36,69.0,M-SC,17.62,149733.0,513.0,143669.0,0.84,0.34,95.95,96.62,208.0,0.0,1.08,62.04,XY25,NTT,FINANCE
59,SAIL,228.0,42.74,48.0,M-MC,12.19,224721.0,-241.0,167889.0,0.16,-0.11,74.71,74.53,192.0,-0.0,1.62,32.11,XY24,BTT,STEEL


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,UJJIVANSFB,60.0,108.87,61.0,H-SC,11.93,132030.0,-10449.0,29971.0,0.68,-7.33,22.70,13.70,163.0,-0.35,0.95,55.83,OX40N,NTT,BANKS
76,TTKPRESTIG,770.0,96.80,50.0,M-SC,1.65,84744.0,-16033.0,16127.0,0.54,-15.91,19.03,0.09,245.0,-0.99,0.61,11.33,OX40N,NTT,DURABLES
44,KANSAINER,340.0,-67.61,53.0,H-SC,1.78,223263.0,-46404.0,82741.0,0.98,-17.21,37.06,13.47,138.0,-0.56,1.60,13.52,XY24,NTT,PAINTS
17,CERA,9475.0,-22.91,45.0,H-SC,2.18,140450.0,-35453.0,77472.0,0.88,-20.15,55.16,23.89,149.0,-0.46,1.01,21.30,OX40N,NTT,CERAMICS
71,TATAELXSI,9161.0,-24.38,45.0,H-MC,7.16,102296.0,-29539.0,71761.0,0.31,-22.41,70.15,32.03,98.0,-0.41,0.74,14.58,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,RECLTD,446.00,47.87,51.0,M-LC,6.14,206992.0,3822.0,38314.0,0.15,1.88,18.51,20.74,55.0,0.10,1.49,7.53,XY25,NTT,FINANCE
31,HINDZINC,730.22,36.83,69.0,M-LC,9.54,223329.0,18253.0,94312.0,0.04,8.90,42.23,54.89,52.0,0.19,1.61,35.14,X5K,ATH,METALS
78,UNIONBANK,163.00,-11.63,57.0,M-LC,8.77,158828.0,17988.0,26508.0,0.18,12.77,16.69,31.59,66.0,0.68,1.14,42.64,XY24,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.85,154938.0,3993.0,105466.0,-0.12,2.65,68.07,72.51,53.0,0.04,1.11,8.83,XR,NTT,IT
11,BANKINDIA,190.00,-24.22,64.0,H-MC,13.07,195157.0,15349.0,97071.0,0.44,8.54,49.74,62.52,88.0,0.16,1.40,45.53,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
31,HINDZINC,730.22,36.83,69.0,M-LC,9.54,223329.0,18253.0,94312.0,0.04,8.90,42.23,54.89,52.0,0.19,1.61,35.14,X5K,ATH,METALS
11,BANKINDIA,190.00,-24.22,64.0,H-MC,13.07,195157.0,15349.0,97071.0,0.44,8.54,49.74,62.52,88.0,0.16,1.40,45.53,XR,NTT,BANKS
85,WIPRO,420.00,-11.98,55.0,M-LC,5.85,154938.0,3993.0,105466.0,-0.12,2.65,68.07,72.51,53.0,0.04,1.11,8.83,XR,NTT,IT
36,INDIAMART,4810.62,-55.01,42.0,H-SC,8.08,125414.0,2078.0,129553.0,1.57,1.69,103.30,106.72,119.0,0.02,0.90,25.46,AR,ATH,MISC
84,WHIRLPOOL,2270.00,-46.09,47.0,M-SC,4.29,91200.0,-298.0,79052.0,2.79,-0.33,86.68,86.07,223.0,-0.00,0.66,32.44,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,13.0,X-LC,5.44,143022.0,-133501.0,243566.0,0.81,-48.28,170.30,39.81,54.0,-0.55,1.03,0.81,XY24,NTT,AUTO
28,HAVELLS,2069.16,-7.88,35.0,X-MC,1.86,228446.0,-19285.0,94348.0,0.01,-7.78,41.30,30.30,92.0,-0.20,1.64,4.47,X40,ATH,ELECTRICAL
51,QUESS,986.00,-47.11,36.0,X-SC,38.50,52472.0,-12534.0,163461.0,0.54,-19.28,311.52,232.18,198.0,-0.08,0.38,2.33,XY24,NTT,MISC
34,ICICIPRULI,790.00,-22.94,40.0,X-MC,2.20,132594.0,-3194.0,45944.0,-0.22,-2.35,34.65,31.48,107.0,-0.07,0.95,9.48,X40,ATH,INSURANCE
12,BATAINDIA,2096.00,-36.44,41.0,X-SC,4.33,95068.0,-33602.0,78897.0,1.93,-26.11,82.99,35.20,219.0,-0.43,0.68,9.12,X40,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SIEMENS,4671.50,-2.61,46.0,H-LC,0.64,156680.0,-29415.0,76899.0,0.48,-15.81,49.08,25.51,15.0,-0.38,1.13,15.46,AR,ATH,ELECTRICAL
83,VOLTAS,1530.00,0.88,57.0,X-MC,1.25,211695.0,19953.0,17804.0,-0.06,10.41,8.41,19.69,99.0,1.12,1.52,17.91,XY25,NTT,AC
28,HAVELLS,2069.16,-7.88,35.0,X-MC,1.86,228446.0,-19285.0,94348.0,0.01,-7.78,41.30,30.30,92.0,-0.20,1.64,4.47,X40,ATH,ELECTRICAL
33,ICICIGI,2260.25,-13.38,67.0,X-MC,2.08,149110.0,12977.0,18147.0,-0.26,9.53,12.17,22.86,91.0,0.72,1.07,23.35,X40,ATH,INSURANCE
49,NESTLEIND,1377.00,-6.57,64.0,X-LC,2.41,287516.0,22090.0,34703.0,0.60,8.32,12.07,21.40,11.0,0.64,2.07,15.52,XY25,NTT,FMCG


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-54.49,13.0,X-LC,5.44,143022.0,-133501.0,243566.0,0.81,-48.28,170.30,39.81,54.0,-0.55,1.03,0.81,XY24,NTT,AUTO
51,QUESS,986.00,-47.11,36.0,X-SC,38.50,52472.0,-12534.0,163461.0,0.54,-19.28,311.52,232.18,198.0,-0.08,0.38,2.33,XY24,NTT,MISC
73,TCS,4389.96,-30.29,44.0,X-LC,12.50,277667.0,-68289.0,135002.0,-0.54,-19.74,48.62,19.28,1.0,-0.51,2.00,2.65,X40,ATH,IT
41,ITC,452.00,-39.19,45.0,X-LC,2.70,195844.0,-4294.0,25186.0,0.15,-2.15,12.86,10.44,4.0,-0.17,1.41,3.30,X40,NTT,FMCG
21,COLPAL,3767.14,-4.32,44.0,X-MC,7.90,223610.0,-39755.0,153106.0,0.40,-15.10,68.47,43.04,84.0,-0.26,1.61,3.81,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,QUESS,986.00,-47.11,36.0,X-SC,38.50,52472.0,-12534.0,163461.0,0.54,-19.28,311.52,232.18,198.0,-0.08,0.38,2.33,XY24,NTT,MISC
55,RELAXO,1176.00,-39.30,44.0,X-SC,4.00,82814.0,-62346.0,138275.0,-0.34,-42.95,166.97,52.31,136.0,-0.45,0.60,10.85,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-36.44,41.0,X-SC,4.33,95068.0,-33602.0,78897.0,1.93,-26.11,82.99,35.20,219.0,-0.43,0.68,9.12,X40,NTT,FOOTWEAR
16,CAMS,5211.76,-4.50,53.0,X-SC,0.99,108693.0,6689.0,37238.0,1.19,6.56,34.26,43.06,122.0,0.18,0.78,24.80,X40N,ATH,MISC
32,HONAUT,58357.33,-23.00,45.0,X-SC,2.39,108435.0,-19503.0,66633.0,-0.03,-15.24,61.45,36.84,143.0,-0.29,0.78,11.01,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.96,-30.29,44.0,X-LC,12.50,277667.0,-68289.0,135002.0,-0.54,-19.74,48.62,19.28,1.0,-0.51,2.00,2.65,X40,ATH,IT
39,INFY,2275.00,-19.96,45.0,X-LC,7.55,312045.0,-843.0,172530.0,-0.64,-0.27,55.29,54.87,3.0,-0.00,2.24,6.36,X40,BTT,IT
41,ITC,452.00,-39.19,45.0,X-LC,2.70,195844.0,-4294.0,25186.0,0.15,-2.15,12.86,10.44,4.0,-0.17,1.41,3.30,X40,NTT,FMCG
82,VBL,671.64,-19.89,47.0,X-LC,10.67,288536.0,-27306.0,139969.0,1.55,-8.65,48.51,35.67,5.0,-0.20,2.07,4.01,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,52.0,H-LC,7.24,250656.0,-10963.0,130166.0,0.94,-4.19,51.93,45.57,7.0,-0.08,1.80,6.66,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,6755.56,58.0,L-SC,6.17,75225.0,-18585.0,95205.0,2.16,-19.81,126.56,81.67,269.0,-0.20,0.54,45.68,XR,NTT,CERAMICS
20,COFFEEDAY,80.00,-43.66,52.0,L-SC,28.80,82752.0,-30797.0,70844.0,1.15,-27.12,85.61,35.27,268.0,-0.43,0.59,102.54,XR,NTT,HOTELS
48,MASFIN,398.61,-20.96,41.0,H-SC,9.46,90045.0,-7935.0,29535.0,-0.32,-8.10,32.80,22.05,152.0,-0.27,0.65,30.89,XR,ATH,FINANCE
77,UJJIVANSFB,60.00,108.87,61.0,H-SC,11.93,132030.0,-10449.0,29971.0,0.68,-7.33,22.70,13.70,163.0,-0.35,0.95,55.83,OX40N,NTT,BANKS
68,SURYODAY,240.00,57.89,55.0,H-SC,10.31,145564.0,-33507.0,99231.0,0.45,-18.71,68.17,36.71,135.0,-0.34,1.05,44.43,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAMMAANCAP,326.00,-186.36,69.0,M-SC,17.62,149733.0,513.0,143669.0,0.84,0.34,95.95,96.62,208.0,0.00,1.08,62.04,XY25,NTT,FINANCE
68,SURYODAY,240.00,57.89,55.0,H-SC,10.31,145564.0,-33507.0,99231.0,0.45,-18.71,68.17,36.71,135.0,-0.34,1.05,44.43,XR,NTT,BANKS
4,ANGELONE,3033.00,15.31,69.0,X-SC,9.36,204090.0,13084.0,44614.0,1.79,6.85,21.86,30.21,157.0,0.29,1.47,29.16,X40N,NTT,FINANCE
31,HINDZINC,730.22,36.83,69.0,M-LC,9.54,223329.0,18253.0,94312.0,0.04,8.90,42.23,54.89,52.0,0.19,1.61,35.14,X5K,ATH,METALS
63,SHALBY,327.00,1179.11,55.0,M-SC,22.97,173874.0,-7269.0,52736.0,2.47,-4.01,30.33,25.10,235.0,-0.14,1.25,39.34,XY24,NTT,HEALTHCARE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.48
1,25,43.95
2,50,75.81


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.54
LC    33.65
MC    21.83
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.64
X40      14.73
XY25     12.16
XR       11.00
X40N     10.20
AR        8.23
OX40N     7.67
X200      1.82
X5K       1.61
SR        1.14
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.62
X-LC    22.52
X-MC    16.08
M-SC    12.75
M-LC     5.35
H-LC     4.75
X-SC     4.69
H-MC     3.83
M-MC     1.62
L-SC     1.48
L-LC     1.03
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.07,-4.82,39.37
IT,12.93,-16.82,80.00
FINANCE,12.24,-11.13,61.50
MISC,6.92,-19.48,85.68
BANKS,6.41,-11.78,70.59
PAINTS,5.57,-19.08,36.84
ELECTRICAL,5.35,-10.64,49.45
INSURANCE,3.92,-1.18,39.00
AUTO,2.85,-45.15,114.54


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3309752.0,22
XR,1379145.0,14
AR,1185064.0,9
X40,788632.0,10
X40N,608573.0,8
OX40N,582720.0,10
XY25,493301.0,8
SR,263509.0,2
X5K,94312.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3437014.0,25
M-SC,1269771.0,15
X-LC,1107599.0,13
X-MC,1087338.0,11
X-SC,529118.0,6
H-MC,347874.0,3
H-LC,278732.0,3
M-LC,264600.0,4
L-SC,255264.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1193102.0      6
           XR         822748.0      7
           AR         816718.0      5
M-SC       XY24       686058.0      6
X-MC       XY24       585969.0      4
X-LC       X40        495935.0      5
           XY24       307723.0      2
X-SC       X40N       286760.0      4
H-SC       OX40N      270467.0      4
           SR         263509.0      2
X-MC       X40        213800.0      4
H-LC       AR         207065.0      2
X-LC       X40N       205154.0      3
H-MC       XY24       179042.0      1
X-MC       XY25       170910.0      2
M-MC       XY24       167889.0      1
L-SC       XR         166049.0      2
X-SC       XY24       163461.0      1
M-SC       AR         161281.0      2
           OX40N      151277.0      4
           XY25       143669.0      1
           XR         127486.0      2
X-MC       X40N       116659.0      1
M-LC       XR         105466.0      1
X-LC       XY25        98787.0      3
H-MC       XR          97071.0      1
M-LC       X5K         94312.0      1
L-SC       OX40N       89215.0      1
X-SC       X40         78897.0      1
H-MC       OX40N       71761.0      1
H-LC       X200        71667.0      1
H-SC       MH          70470.0      1
L-MC       XR          60325.0      1
L-LC       XY25        41621.0      1
M-LC       XY25        38314.0      1
           XY24        26508.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 56.0 seconds
